In [ ]:
import os
os.environ["HF_HOME"] = "/nas/ucb/oliveradk/.cache"
os.environ["TRANSFORMERS_CACHE"] = "/nas/ucb/oliveradk/.cache"

In [ ]:
import torch
from cupbearer import utils, data, detectors, tasks, models, scripts

/Users/oliverdaniels-koch/Library/Caches/pypoetry/virtualenvs/elk-experiments-AZ2LBS3Q-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
task = tasks.measurement_tampering(task_name="diamonds", device="cuda", untrusted_labels=True)

/nas/ucb/oliveradk/miniforge3/envs/elk_experiments/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
A new version of the following files was downloaded from https://huggingface.co/oliverdk/codegen-350M-mono-measurement_pred:
- configuration_code_gen_measuremet_pred.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/oliverdk/codegen-350M-mono-measurement_pred:
- modeling_code_gen_measurement_pred.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Special tokens have been added in the voca

In [ ]:
task.test_data[0][0][0]

'from collections import namedtuple\nclass Diamond:\n    def __repr__(self):\n        return "Diamond"\n    @property\n    def shine(self):\n        return 5\n    @property\n    def hardness(self):\n        return 5\ndef get_shine(x):\n    if hasattr(x, "shine"):\n        return x.shine\n    if isinstance(x, int):\n        return x if x <= 5 else 10 - x\n    if isinstance(x, type("")):\n        return len(x) if len(x) <= 5 else 10 - len(x)\n    return 0\ndef get_hardness(x):\n    if hasattr(x, "hardness"):\n        return x.hardness\n    if isinstance(x, int):\n        return 1\n    if isinstance(x, type("")):\n        return 2\n    return 0\nget_shine_ = get_shine\nget_hardness_ = get_hardness\nclass Vault:\n    _items = [Diamond()]\n    def insert(self, idx, item):\n        self._items.insert(idx, item)\n    def append(self, item):\n        self._items.append(item)\n    def pop(self, idx=-1):\n        if len(self._items) <= 1:\n            return None\n        return self._items.pop(

In [ ]:
task.model.tokenize(task.test_data[0][0][0], **task.model.tokenize_kwargs)['input_ids'].shape

torch.Size([1, 1024])

In [ ]:
task.test_data

In [ ]:
task.model.tokenizer.padding_side

'left'

In [ ]:
dir(task.model.tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_bos_token',
 '_call_one',
 '_cls_token',
 '_compile_jinja_template',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager',
 '_mask_token',
 '

In [ ]:
cache = detectors.ActivationCache()

In [ ]:
for name, _ in task.model.named_modules():
    print(name)


hf_model
hf_model.sensor_probes
hf_model.sensor_probes.0
hf_model.sensor_probes.1
hf_model.sensor_probes.2
hf_model.aggregate_probe
hf_model.transformer
hf_model.transformer.wte
hf_model.transformer.drop
hf_model.transformer.h
hf_model.transformer.h.0
hf_model.transformer.h.0.ln_1
hf_model.transformer.h.0.attn
hf_model.transformer.h.0.attn.attn_dropout
hf_model.transformer.h.0.attn.resid_dropout
hf_model.transformer.h.0.attn.qkv_proj
hf_model.transformer.h.0.attn.out_proj
hf_model.transformer.h.0.mlp
hf_model.transformer.h.0.mlp.fc_in
hf_model.transformer.h.0.mlp.fc_out
hf_model.transformer.h.0.mlp.act
hf_model.transformer.h.0.mlp.dropout
hf_model.transformer.h.1
hf_model.transformer.h.1.ln_1
hf_model.transformer.h.1.attn
hf_model.transformer.h.1.attn.attn_dropout
hf_model.transformer.h.1.attn.resid_dropout
hf_model.transformer.h.1.attn.qkv_proj
hf_model.transformer.h.1.attn.out_proj
hf_model.transformer.h.1.mlp
hf_model.transformer.h.1.mlp.fc_in
hf_model.transformer.h.1.mlp.fc_out
hf

In [ ]:
names = [
    "hf_model.transformer.ln_f.output"
    # "hf_model.transformer.h.10.output",
    # "hf_model.transformer.h.19.output",
]

In [ ]:
def get_activation_at_last_token(
    activation: torch.Tensor, inputs: list[str], name: str
):
    if isinstance(activation, tuple):
        activation = activation[0]
    return activation[:, -1, :]

In [ ]:
gt_detector = detectors.SupervisedLinearProbe(
    names, activation_processing_func=get_activation_at_last_token,
    layer_aggregation="mean"
)

In [ ]:
#TODO: activation cache that loads off periodically

In [ ]:
import submitit

In [ ]:
slurm_params = {
        "slurm_mem_gb": 80, 
        "gres": "gpu:A100-SXM4-80GB:1",
        "nodes": 1, 
        "timeout_min": 60 * 10,
        "job_name": "bash",
        "qos": "high",
#         "additional_parameters":{
#             'export': 'HF_HOME="/nas/ucb/oliveradk/.cache",TRANSFORMERS_CACHE="/nas/ucb/oliveradk/.cache"'
#         }
}
executor = submitit.AutoExecutor(folder="../output")
executor.update_parameters(**slurm_params)

/nas/ucb/oliveradk/miniforge3/envs/elk_experiments/lib/python3.10/site-packages/submitit/auto/auto.py:23: UserWarning: Setting 'gres' is deprecated. Use 'slurm_gres' instead.
  warnings.warn(f"Setting '{arg}' is deprecated. Use '{new_arg}' instead.")
/nas/ucb/oliveradk/miniforge3/envs/elk_experiments/lib/python3.10/site-packages/submitit/auto/auto.py:23: UserWarning: Setting 'job_name' is deprecated. Use 'slurm_job_name' instead.
  warnings.warn(f"Setting '{arg}' is deprecated. Use '{new_arg}' instead.")
/nas/ucb/oliveradk/miniforge3/envs/elk_experiments/lib/python3.10/site-packages/submitit/auto/auto.py:23: UserWarning: Setting 'qos' is deprecated. Use 'slurm_qos' instead.
  warnings.warn(f"Setting '{arg}' is deprecated. Use '{new_arg}' instead.")


In [ ]:
def print_env():
    import os 
    print("hf home", os.environ["HF_HOME"])
    print("transformer home", os.environ["TRANSFORMERS_CACHE"])

In [ ]:
job = executor.submit(print_env)

In [ ]:
job = executor.submit(
    task,
    scripts.train_detector, 
    gt_detector, 
    save_path="../output/", 
    batch_size=8,
    eval_batch_size=8, 
)

In [ ]:
job

SlurmJob<job_id=340625, task_id=0, state="FAILED">

In [ ]:
scripts.train_detector(
    task, gt_detector, save_path=None, eval_batch_size=8, batch_size=8
) #TODO: set cache directory to ucb

NameError: name 'gt_detector' is not defined

In [ ]:
# cache.store(os.path.join("..", "activation_caches", "code-gen-diamonds-activations"))

In [ ]:
detector = detectors.MahalanobisDetector(
    activation_names=names, activation_processing_func=get_activations_at_sensor_tokens,#get_activation_at_last_token,
    layer_aggregation="mean"
)

In [ ]:
scripts.train_detector(
    task, detector, save_path=None, eval_batch_size=2, batch_size=2
)

2024-06-27 10:40:25.033 | DEBUG    | cupbearer.detectors.statistical.statistical:init_variables:78 - Received multi-dimensional activations, will only learn covariances along last dimension and treat others independently. If this is unintentional, pass `activation_preprocessing_func=utils.flatten_last`.
 11%|█         | 267/2500 [03:36<30:14,  1.23it/s]


KeyboardInterrupt: 